In [ ]:
# 2.3 ICESat-2 extraction, filtering
#### This notebook uses a DPS job to run extract_atl08.py to convert h5's to csv's, then appends all csv's into a pandas geodataframe.
#### Returns: a pandas geodataframe that should hold the entire set of ATL08 data for this project
###### Notes:
###### 

In [ ]:
import geopandas as gpd
from geopandas import GeoDataFrame
from geopandas.tools import sjoin
import pandas as pd
import glob
import os

# Run extract_at08.py as a DPS job (see nb 1.3 for template of how this can be done)
DPS_OUTPUT_DIR = '/projects/jabba/dps_output'

In [ ]:
# DPS SUBMISSION ---- NOT YET WORKING ----

# Here is where I submit a job 
# identified with 'algo_id' (in yaml file)
# that specifies a bash script (/jabba/code/dps_code/run_extract_atl08.sh) 
# that will call the 'algorithm' (extract_atl08.py)
# that takes an granule URL specifying a .H5 file : python /jabba/code/airborne/lvis_metrics.R ${GRANULE_URL} -output ??

# Uses granule list from nb 2.1
# CHANGE the submitJob args!
for granule in granules_list_ATL08:
    submit_result = maap.submitJob(identifier="ice2", algo_id="ice2_h5tocsv_ubuntu", 
                               version="master", 
                               username="r2d2", 
                               ice2_granules=granule)
    print(submit_result)

In [ ]:
# Merge all CSV from extract_at08.py into a geodataframe

os.chdir(DPS_OUTPUT_DIR)
extension = 'csv'
all_atl08_csvs = [i for i in glob.glob('ATL08*.{}'.format(extension))]

# Merge all files in the list
atl08 = pd.concat([pd.read_csv(f) for f in all_atl08_csvs ])

# Probably not necessary
atl08.to_csv( "atl08_merged.csv", index=False, encoding='utf-8-sig')

# CREATE GEOPANDAS DATAFRAME WITH SPATIAL REF
#atl08 = GeoDataFrame(atl08, geometry=gpd.points_from_xy(atl08.lon, atl08.lat), crs='epsg:4326')
atl08 = GeoDataFrame(atl08, crs='epsg:4326',
    geometry=[Point(float(X), float(Y), float(Z)) for X,Y,Z in zip(atl08.lon, atl08.lat, atl08.h_te_best)])

print(f"There are {atl08.shape[0]} observations in this geodataframe.")
atl08.head()